In [2]:
import iris
import iris.plot as iplt
import numpy as np
from iris.coords import DimCoord
from iris.coord_systems import TransverseMercator,GeogCS
from iris.cube import Cube
from cf_units import Unit
import cf_units
import os
import glob
from pyproj import Proj, transform
import sys
import warnings
import multiprocessing as mp
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

ems_hourly = ['01', '04', '06', '07', '08', '09', '10', '11', '12', '13', '15']
ems_30mins = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc013',  'bc015',  'bc016', 'bc017', 'bc018', 'bc012']
yrs_range = '1980_2001'


def mask_cube(cube, gb_mask):
    masked_cube_data = cube * gb_mask[np.newaxis, :, :]

    # APPLY THE MASK
    reshaped_mask = np.tile(gb_mask, (cube.shape[0], 1, 1))
    reshaped_mask = reshaped_mask.astype(int)
    reversed_array = ~reshaped_mask.astype(bool)

    # Mask the cube
    masked_cube = iris.util.mask_cube(cube, reversed_array)
    
    return masked_cube

### Get the mask
gb_mask_2km = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_2.2km_GB_Mask.npy")
gb_mask_2km_bng = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_2.2km_bng_GB_Mask.npy")
gb_mask_12km_wgs84 = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_wgs84_GB_Mask.npy")
gb_mask_12km = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_GB_Mask.npy")
gb_mask_nimrod = np.load('/nfs/a319/gy17m2a/PhD/datadir/nimrod_1km_GB_Mask_[1100,200].npy')
# gb_mask_12km_bng = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_bng_GB_Mask.npy")

In [4]:
##########################################################################################
#########################################################################################
# Define variables and set up environment
##########################################################################################
##########################################################################################
root_fp = "/nfs/a319/gy17m2a/"
os.chdir(root_fp)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'PhD/Scripts/DataProcessing/Regridding')
from Regridding_functions import *
# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

gb_gdf = create_gb_outline({'init' :'epsg:3857'})

# Load UKCP18 12km model data to use in regriddding
file_model_12km=f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/12km/01/pr_rcp85_land-rcm_uk_12km_01_day_19801201-19901130.nc'
cube_12km=iris.load_cube(file_model_12km)

file_model_2_2km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_original/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km =iris.load_cube(file_model_2_2km)

file_model_2_2km_bng ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng/01/1980_2001/bng_pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km_bng =iris.load_cube(file_model_2_2km_bng)

file_model_2_2km_bng_30mins = '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_original/bc005/2002_2020/bc005a.pr200001.nc'
file_model_2_2km_bng_30mins = '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_original/bc005/2002_2020/bc005a.pr200001.nc'
cube_2km_30mins = iris.load_cube(file_model_2_2km_bng_30mins)
# # Trim ensemble member dimension
cube_2km_30mins = trim_to_bbox_of_region_regriddedobs(cube_2km_30mins, gb_gdf)
# # Transform to BNG
cube_2km_30mins_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km_30mins.copy())

# remove ensemble member dimension
cube_2km = cube_2km[0,:,:,:]
cube_12km = cube_12km[0,:,:,:]

yrs_range = '1980_2001'

in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)

In [5]:
lsm = iris.load("/nfs/a319/gy17m2a/PhD/datadir/lsm_land-cpm_BI_5km.nc")[0]
lsm_2km = lsm.regrid(cube_2km_bng, iris.analysis.AreaWeighted(mdtol=0.9)) 
lsm_2km = lsm.regrid(cube_2km_bng, iris.analysis.Nearest()) 

broadcasted_lsm_2km_data = np.broadcast_to(lsm_2km.data.data, cube_2km_bng.shape)
broadcasted_lsm_2km_int = broadcasted_lsm_2km_data.astype(int)
broadcasted_lsm_2km_data_reversed = ~broadcasted_lsm_2km_data.astype(bool)

broadcasted_lsm_2km_30mins_data = np.broadcast_to(lsm_2km.data.data, cube_2km_30mins_bng.shape)
broadcasted_lsm_2km_30mins_data_reversed = ~broadcasted_lsm_2km_30mins_data.astype(bool)

## Transform cube 2.2km to BNG and then to 12km - masking first

In [25]:
for em in ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15']:
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_original/{em}/{yrs_range}/")

    output_fp_bng = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng/{em}/{yrs_range}/"
    output_fp_bng_masked = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng_masked/{em}/{yrs_range}/"
    output_fp_bng_regridded_masked = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng_regridded_12km_masked/{em}/AreaWeighted/{yrs_range}/"
    
    # Create directories if they don't exist
    for filepath in [output_fp_bng, output_fp_bng_masked,output_fp_bng_regridded_masked ]:
        if not os.path.isdir(filepath):
            os.makedirs(filepath)
    
    for filename in np.sort(glob.glob("*")):     
            if filename[47:49] in ['06', '07', '08']:
                if os.path.isfile(output_fp_bng +  f"bng_{filename}"):
                    print("already exist")                    
                else:
                    print(filename)
                    # Load the data
                    cube_2km = iris.load(filename)[0]
                    # Trim ensemble member dimension
                    cube_2km = cube_2km[0,:,:,:]
                    # Trim
                    cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
                    # Transform to BNG
                    cube_2km_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km.copy())
                    # Mask to GB
                    cube_2km_bng_masked = iris.util.mask_cube(cube_2km_bng.copy(), broadcasted_lsm_2km_data_reversed)
                    # Regrid to 12km
                    cube_2km_bng_masked_regridded_12km = cube_2km_bng_masked.regrid(cube_12km, iris.analysis.AreaWeighted(mdtol=0.8)) 
                    # Save 
                    iris.save(cube_2km_bng, output_fp_bng +  f"bng_{filename}")     
                    iris.save(cube_2km_bng_masked, output_fp_bng +  f"bng_{filename}")
                    iris.save(cube_2km_bng_masked_regridded_12km, output_fp_bng_regridded_masked +  f"bng_rg_{filename}") 
                    

already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
alread

pr_rcp85_land-cpm_uk_2.2km_11_1hr_19990601-19990630.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_19990701-19990730.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_19990801-19990830.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20000601-20000630.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20000701-20000730.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20000801-20000830.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20010601-20010630.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20010701-20010730.nc
pr_rcp85_land-cpm_uk_2.2km_11_1hr_20010801-20010830.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19810601-19810630.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19810701-19810730.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19810801-19810830.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19820601-19820630.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19820701-19820730.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19820801-19820830.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19830601-19830630.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19830701-19830730.nc
pr_rcp85_land-cpm_uk_2.2km_12_1hr_19830801-19830830.nc
pr_rcp85_l

pr_rcp85_land-cpm_uk_2.2km_15_1hr_19850801-19850830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19860601-19860630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19860701-19860730.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19860801-19860830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19870601-19870630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19870701-19870730.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19870801-19870830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19880601-19880630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19880701-19880730.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19880801-19880830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19890601-19890630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19890701-19890730.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19890801-19890830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19900601-19900630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19900701-19900730.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19900801-19900830.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19910601-19910630.nc
pr_rcp85_land-cpm_uk_2.2km_15_1hr_19910701-19910730.nc
pr_rcp85_l

## Regrid 30 mins UKCP18 data to 12km - DONE
For this need to use version of the regridding cube which is trimmed to the leeds-at-centre region, as the 30 mins data is already clipped to this extent

In [ ]:
for em in ems_30mins:
    print(em)
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km/{em}/2002_2020/")
    # establish paths to directories
    output_fp_bng = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{em}/{yrs_range}/"
    output_fp_bng_regridded = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng_regridded_12km/{em}/AreaWeighted/{yrs_range}/"
    # create the directories
    if not os.path.isdir(output_fp_bng):
        os.makedirs(output_fp_bng)
    if not os.path.isdir(output_fp_bng_regridded):
        os.makedirs(output_fp_bng_regridded)      
    # loop through the files
    for filename in np.sort(glob.glob("*")):   
        print(filename)
        if not os.path.isfile(output_fp_bng_regridded +  f"bng_rg_{filename}"):
            print("creating")
            
            # Load the data
            cube_2km = iris.load(filename)[0]
            # Trim
            cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
            # Transform to BNG
            cube_2km_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km.copy())
            # Mask to GB
            cube_2km_bng_masked = iris.util.mask_cube(cube_2km_bng.copy(), broadcasted_lsm_2km_30mins_data_reversed)
            # Regrid to 12km
            cube_2km_bng_masked_regridded_12km = cube_2km_bng_masked.regrid(cube_12km, iris.analysis.AreaWeighted(mdtol=0.8)) 
            # Save 
            iris.save(cube_2km_bng_masked_regridded, output_fp_bng_regridded +  f"bng_rg_{filename}")     
            iris.save(cube_2km_bng_masked, output_fp_bng +  f"bng_{filename}")
            
        else:
            print("already exists")    

bc005
bc005a.pr200001.nc
creating
bc005a.pr200006.nc
creating
bc005a.pr200007.nc
creating
bc005a.pr200008.nc
creating
bc005a.pr200101.nc
creating
bc005a.pr200102.nc
creating
bc005a.pr200103.nc
creating
bc005a.pr200104.nc
creating
bc005a.pr200105.nc
creating
bc005a.pr200106.nc
creating
bc005a.pr200107.nc
creating
bc005a.pr200108.nc
creating
bc005a.pr200109.nc
creating
bc005a.pr200110.nc
creating
bc005a.pr200111.nc
creating
bc005a.pr200112.nc
creating
bc005a.pr200201.nc
creating
bc005a.pr200202.nc
creating
bc005a.pr200203.nc
creating
bc005a.pr200204.nc
creating
bc005a.pr200205.nc
creating
bc005a.pr200206.nc
creating
bc005a.pr200207.nc
creating
bc005a.pr200208.nc
creating
bc005a.pr200209.nc
creating
bc005a.pr200210.nc
creating
bc005a.pr200211.nc
creating
bc005a.pr200212.nc
creating
bc005a.pr200301.nc
creating
bc005a.pr200302.nc
creating
bc005a.pr200303.nc
creating
bc005a.pr200304.nc
creating
bc005a.pr200305.nc
creating
bc005a.pr200306.nc
creating
bc005a.pr200307.nc
creating
bc005a.pr20030

## Regrid NIMROD - DONE

In [21]:
file_nimrod_1km =f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Unfiltered/2018/metoffice-c-band-rain-radar_uk_20180601_30mins.nc"
nimrod_1km =iris.load_cube(file_nimrod_1km)
nimrod_1km= trim_to_bbox_of_region_obs(nimrod_1km, gb_gdf, 'projection_y_coordinate', 'projection_x_coordinate')

lsm_1km = lsm.regrid(nimrod_1km, iris.analysis.Nearest()) 

In [ ]:
for year in range(2006,2025):
    print(year)
    # Change directory to be for correct year
    os.chdir(f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Unfiltered/{year}")
    # Define filepaths to save files to
    output_dir_2_2km = f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_2.2km/Unfiltered/AreaWeighted/{year}/"
    # Create these directories if they don't exist already
    if not os.path.isdir(output_dir_2_2km):
        os.makedirs(output_dir_2_2km)
    # Loop through all the diles in the 1km folder    
    for filename in sorted(glob.glob("*")):
        print(filename)

        # Create version of filename specifying it is regridded
        filename_to_save_to = f"rg_{filename}"

        # 2km regridding
        print("JJA")
        cube = iris.load(filename, in_jja)[0] 
        # Fill in missing bounds
        cube.coord('projection_y_coordinate').guess_bounds()
        cube.coord('projection_x_coordinate').guess_bounds()
        # Align small rounding error in coordinates
        cube.coord('projection_x_coordinate').coord_system = cube_2km_bng.coord('projection_x_coordinate').coord_system
        cube.coord('projection_y_coordinate').coord_system = cube_2km_bng.coord('projection_y_coordinate').coord_system

        # Trim 
        cube= trim_to_bbox_of_region_obs(cube, gb_gdf, 'projection_y_coordinate', 'projection_x_coordinate')
        
        # Reshape to match the data (data not always same number of timeslices so eneds redoing each time)
        broadcasted_lsm_1km_data = np.broadcast_to(lsm_1km.data.data, cube.shape)
        broadcasted_lsm_1km_data_reversed = ~broadcasted_lsm_1km_data.astype(bool)
        
        # Apply mask
        cube_masked = iris.util.mask_cube(cube.copy(), broadcasted_lsm_1km_data_reversed)
        
        # Area Weighted
        reg_cube_masked =cube_masked.regrid(cube_2km_bng,iris.analysis.AreaWeighted())    
        print("Regridded")
        # Save 
        iris.save(reg_cube_masked, output_dir_2_2km + filename_to_save_to)    

2006
metoffice-c-band-rain-radar_uk_20060601_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060602_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060603_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060604_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060605_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060606_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060607_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060608_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060609_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060610_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060611_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060612_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060613_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060614_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20060615_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_2006

metoffice-c-band-rain-radar_uk_20070710_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070711_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070712_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070713_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070714_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070715_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070716_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070717_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070718_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070719_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070720_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070721_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070722_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070723_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070724_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20070725_

metoffice-c-band-rain-radar_uk_20080816_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080818_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080819_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080820_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080821_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080822_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080823_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080824_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080825_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080826_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080827_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080828_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080829_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080830_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20080831_30mins.nc
JJA
Regridded
2009
metoffice-c-band-rain-radar_uk_2009

metoffice-c-band-rain-radar_uk_20100703_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100704_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100705_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100706_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100707_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100708_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100709_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100710_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100711_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100712_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100713_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100714_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100715_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100716_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100717_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20100718_

metoffice-c-band-rain-radar_uk_20110808_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110809_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110810_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110811_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110812_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110813_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110814_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110815_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110816_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110817_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110818_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110819_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110820_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110821_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110822_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20110823_

metoffice-c-band-rain-radar_uk_20130614_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130615_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130616_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130617_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130618_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130619_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130620_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130621_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130622_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130623_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130624_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130625_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130626_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130627_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130628_30mins.nc
JJA
Regridded
metoffice-c-band-rain-radar_uk_20130629_

In [ ]:
# fig, ax=plt.subplots(figsize=(20,20))
# iplt.contourf(cube_masked[1,:,:])
# plt.gca().coastlines()

## Regrid CEH-GEAR

### Test method with one file

In [10]:
### Read in reformatted data and set coordinate bounds
# (Might have deleted bit of code from previous versions that do the reformatting, check git history)

cube_1km = iris.load('/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/rf_CEH-GEAR-1hr_199006.nc', in_jja)[0]
cube_1km.coord('projection_y_coordinate').guess_bounds()
cube_1km.coord('projection_x_coordinate').guess_bounds()

### Regrid to 12km using different available methods
cube_aw= cube_1km.regrid(cube_2km_bng, iris.analysis.AreaWeighted()) 
cube_nn= cube_1km.regrid(cube_2km_bng, iris.analysis.Nearest()) 
cube_l= cube_1km.regrid(cube_2km_bng, iris.analysis.Linear()) 

In [18]:
# fig, ax= plt.subplots(figsize=(15,15))
# iplt.contourf(cube_1km[15,:,:])
# plt.gca().coastlines(resolution='10m', color='black', linewidth=0.5);

### Run properly on all files

In [129]:
# contraint so we only bother for JJA data
in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)
# Directory where data will be saved 
dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
# For all files in the directory
for filename in np.sort(glob.glob("/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/*")):
    print(filename)
    filename_to_save_to = f"rg_{filename}"
    # Check if it doesn't already exist
    if not os.path.isfile(dir_to_save_to + filename_to_save_to):
        print("Creating")
        # if it's in JJA
        try:
            cube_1km = iris.load(filename, in_jja)[0] 
            print("JJA")
            # Coordinates dont have bounds
            cube_1km.coord('projection_y_coordinate').guess_bounds()
            cube_1km.coord('projection_x_coordinate').guess_bounds()
            # Regrid with area weighted
            cube_aw= cube_1km.regrid(cube_12km, iris.analysis.AreaWeighted()) 
            # File path to save to
            dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
            filename_to_save_to = f"rg_{filename}"
            iris.save(cube_aw, dir_to_save_to + filename_to_save_to)
        except:
            print("Not JJA")
    else:
        print("already exists")

rf_CEH-GEAR-1hr_199001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199006.nc
Creating
JJA
rf_CEH-GEAR-1hr_199007.nc
already exists
rf_CEH-GEAR-1hr_199008.nc
Creating
JJA
rf_CEH-GEAR-1hr_199009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199106.nc
already exists
rf_CEH-GEAR-1hr_199107.nc
Creating
JJA
rf_CEH-GEAR-1hr_199108.nc
Creating
JJA
rf_CEH-GEAR-1hr_199109.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199110.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199111.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199112.nc
Creat

Exception ignored in: <function CFReader.__del__ at 0x7f64deb683b0>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


rf_CEH-GEAR-1hr_199908.nc
Creating
JJA
rf_CEH-GEAR-1hr_199909.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199910.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199911.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199912.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200006.nc
Creating
JJA
rf_CEH-GEAR-1hr_200007.nc
Creating
JJA
rf_CEH-GEAR-1hr_200008.nc
Creating
JJA
rf_CEH-GEAR-1hr_200009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200106.nc
Creating
JJA
rf_CEH-GEAR-1hr_200107.nc
Creat